# Example of slurm job creation and submission

## Structure

1. [Importing the job module](#1-importing-the-job-module)

2. [Creating the Slurm Job](#2-creating-the-slurm-job)

    2.1 [Creating the Job with manual selection of cores, memory, and walltime](#21-creating-the-job-with-manual-selection-of-cores-memory-and-walltime)

    2.2 [Creating the Job with exclusive node access](#22-creating-the-job-with-exclusive-node-access)

    2.3 [Creating the Job with maxumum availibale resources per the node](#23-creating-the-job-with-maxumum-availibale-resources-per-the-node)

    2.4 [Redirecting the SLURM output to /any/path/you/want](#24-redirecting-the-slurm-output-to-anypathyouwant)

3. [Creating and Submitting the Job to the SLURM queue on different machines](#3-creating-and-submitting-the-job-to-the-slurm-queue-on-lumi)

    3.1 [Specifying Machine Name](#31-specifying-machine-name)

    3.2 [Manual Parameter Specification (Generic Configuration)](#32-manual-parameter-specification-generic-configuration)

    3.3. [Modifying the YAML Configuration File (Configuration Management)](#33-modifying-the-yaml-configuration-file-configuration-management)
    
4. [Canceling the Slurm Job](#4-canceling-the-slurm-job)

    4.1 [Canceling all jobs of the user](#41-cancelling-all-jobs-of-user)

    4.2 [Canceling the specific Job](#42-canceling-specific-job)

## 1. Importing the job module

The `job` module contains the following functions: `squeue`, `job`, `max_resources_per_node`, `scancel`, which allows us to create and operate the Slurm Job

In [1]:
from aqua.slurm import slurm

## 2. Creating the Slurm Job

### 2.1 Creating the Job with manual selection of cores, memory, and walltime

The function `slurm.job()` allows to create a job in the slurm queue. 
It works both on Lumi and Levante.
Some options are available.

In [25]:
slurm.job()

/LUMI_TYKKY_eiW6rEt/miniconda/envs/env1/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41059 instead
  warnings.warn(
2024-05-10 20:17:57 :: slurm :: WARNING  -> #!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p small
#SBATCH -A project_465000454
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G
#SBATCH -t 02:30:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out

/users/nazarova/mambaforge/aqua/bin/python3.11 -m distributed.cli.dask_worker tcp://193.167.209.165:38213 --name dummy-name --nthreads 1 --memory-limit 9.31GiB --nanny --death-timeout 60



We can check the status of created Job in the queue using the function `squeue()`

In [22]:
slurm.squeue()

JOBID      CPUS  NODES ST         NAME                 TIME       START_TIME           DEPENDENCY           PARTITION            MIN_MEMORY          
7078525    1     1     PD         dask-worker          0:00       N/A                  (null)               small                10G                 


0

To cancel all the jobs:

In [23]:
slurm.scancel()

By default, the Job has the following attributes:

- `exclusive=False`: If True, the job will be submitted asking for exclusive access to the node.
- `max_resources=False`: If True, the job will be submitted asking for the maximum resources available on the node.
- `cores=1`: number of cores per socket.
- `memory="10 GB"`: real memory required per node.
- `walltime="02:30:00"`: duration of the allocation.
- `jobs=1`: factor of assignment scaling across multiple nodes,
- `path_to_output="."`: path where log, err and output files are stored,
- `account="project_465000454"`: the submission queue for job and resource allocation,
- `queue="small"`: project account under which the job runs.
 
If you want to use a different amount of cores, memory, wall time, jobs, or a different queue, you can specify it as an argument of function:

In [5]:
slurm.job(cores=8, memory="50 GB", queue="interactive", walltime='00:30:00', jobs=1)

/LUMI_TYKKY_eiW6rEt/miniconda/envs/env1/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39135 instead
  warnings.warn(
2024-05-08 18:34:41 :: slurm :: WARNING  -> #!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p interactive
#SBATCH -A project_465000454
#SBATCH -n 1
#SBATCH --cpus-per-task=8
#SBATCH --mem=47G
#SBATCH -t 00:30:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out

/users/nazarova/mambaforge/aqua/bin/python3.11 -m distributed.cli.dask_worker tcp://193.167.209.165:38017 --name dummy-name --nthreads 2 --memory-limit 11.64GiB --nworkers 4 --nanny --death-timeout 60



In [6]:
slurm.squeue()

JOBID      CPUS  NODES ST         NAME                 TIME       START_TIME           DEPENDENCY           PARTITION            MIN_MEMORY          


0

In [7]:
slurm.scancel(loglevel="INFO")

2024-05-08 18:34:41 :: slurm :: INFO     -> Cancelling all user jobs in the queue


### 2.2. Creating the Job with exclusive node access

The function has an argument `exclusive`, which is False by default.  If we set the argument to True, we will get exclusive access to the node.


In [8]:
slurm.job(exclusive=True)

/LUMI_TYKKY_eiW6rEt/miniconda/envs/env1/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36525 instead
  warnings.warn(


2024-05-08 18:34:41 :: slurm :: WARNING  -> #!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p small
#SBATCH -A project_465000454
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G
#SBATCH -t 02:30:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out
#SBATCH --get-user-env
#SBATCH --exclusive

/users/nazarova/mambaforge/aqua/bin/python3.11 -m distributed.cli.dask_worker tcp://193.167.209.165:35275 --name dummy-name --nthreads 1 --memory-limit 9.31GiB --nanny --death-timeout 60



In [9]:
slurm.squeue()

JOBID      CPUS  NODES ST         NAME                 TIME       START_TIME           DEPENDENCY           PARTITION            MIN_MEMORY          


0

In [10]:
slurm.scancel(Job_ID=7054951, loglevel="INFO")

2024-05-08 18:34:41 :: slurm :: INFO     -> Cancelling the job with ID: 7054951


## Important! 

The `exclusive` argument DOES NOT automatically provide us the maximum available memory, number of cores, and walltime!
It only provide you the exclusive usage of the node, meaning that no other job can run at the same time on the same node.

In [11]:
slurm.job(exclusive=True, cores=32, memory="200 GB", queue = "interactive", walltime='8:00:00', jobs=1)

/LUMI_TYKKY_eiW6rEt/miniconda/envs/env1/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37191 instead
  warnings.warn(
2024-05-08 18:34:41 :: slurm :: WARNING  -> #!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p interactive
#SBATCH -A project_465000454
#SBATCH -n 1
#SBATCH --cpus-per-task=32
#SBATCH --mem=187G
#SBATCH -t 8:00:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out
#SBATCH --get-user-env
#SBATCH --exclusive

/users/nazarova/mambaforge/aqua/bin/python3.11 -m distributed.cli.dask_worker tcp://193.167.209.165:45407 --name dummy-name --nthreads 4 --memory-limit 23.28GiB --nworkers 8 --nanny --death-timeout 60



In [12]:
slurm.squeue()

JOBID      CPUS  NODES ST         NAME                 TIME       START_TIME           DEPENDENCY           PARTITION            MIN_MEMORY          
7056337    32    1     PD         dask-worker          0:00       N/A                  (null)               interactive          187G                
7056336    1     1     PD         dask-worker          0:00       N/A                  (null)               small                10G                 


0

In [13]:
slurm.scancel()

### 2.3. Creating the Job with maxumum availibale resources per the node

The function has an argument `max_resources_per_node`, which is `False` by default. If we set the argument to `True`, the number of cores, memory, and walltime will equal the maximum allowed by the selected queue/partition.

In [26]:
slurm.max_resources_per_node(queue='small')

('99.19921875 GB', '3-00:00:00', '128', '2', '64', '2')

In [20]:
slurm.job(max_resources=True)

/LUMI_TYKKY_eiW6rEt/miniconda/envs/env1/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43235 instead
  warnings.warn(
2024-05-09 17:06:55 :: slurm :: WARNING  -> #!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p small
#SBATCH -A project_465000454
#SBATCH -n 1
#SBATCH --cpus-per-task=128
#SBATCH --mem=93G
#SBATCH -t 3-00:00:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out

/users/nazarova/mambaforge/aqua/bin/python3.11 -m distributed.cli.dask_worker tcp://193.167.209.164:46565 --name dummy-name --nthreads 8 --memory-limit 5.77GiB --nworkers 16 --nanny --death-timeout 60



In [22]:
slurm.squeue()

JOBID      CPUS  NODES ST         NAME                 TIME       START_TIME           DEPENDENCY           PARTITION            MIN_MEMORY          


0

In [21]:
slurm.scancel()

With the argument `max_resources_per_node=True`, the function `max_resources_per_node()` automatically extracts the following information about the resources in any queue:

 - Size of memory per node in Gigabytes: `max_memory`, 

 - Maximum time for any job in the format "days-hours:minutes:seconds: `max_walltime`

 - Number of CPUs per node: `max_cpus`, 

 - Number of sockets per node: `max_sockets`

 - Number of cores per socket: `max_cores`, 
 
 - Number of threads per core: `max_threads`

The function can be also used alone to have info about a specific queue/partition.
For example:

In [21]:
slurm.max_resources_per_node('small')

('99.19921875 GB', '3-00:00:00', '128', '2', '64', '2')

In [22]:
slurm.max_resources_per_node('interactive')

('224.0 GB', '8:00:00', '128', '2', '64', '2')

### 2.4 Redirecting the SLURM output to `/any/path/you/want`

Slurm Job writes by default
    - the errors into `./slurm/logs` directory 
    - the output into `./slurm/output/` directory

If folders `/slurm`,  `/slurm/output`,  `/slurm/logs` do not exist, the function will create them automatically. 

The user can specify a custom path to redirect the SLURM outputs with the `path_to_output` option:

In [ ]:
slurm.job(path_to_output='/any/path/you/want')

## 3. Creating and Submitting the Job to the SLURM queue on different machines

The job() function is designed to submit jobs to a SLURM queue on various machines, utilizing configurations that are either predefined in a YAML configuration file `AQUA/aqua/slurm/config-slurm.yml` or manually specified at the time of the function call. This flexible design allows users to adapt the job submission process according to the specific requirements of different computing environments or projects.

### 3.1. Specifying Machine Name

The user specifies the name of the machine, and the job() function automatically loads the corresponding configuration from a YAML file. This configuration includes all necessary parameters such as memory, core allocation, walltime, and others.

In [ ]:
slurm.job(machine='levante')

### 3.2. Manual Parameter Specification (Generic Configuration)

The user manually inputs all relevant parameters like memory, cores, walltime, etc. This method bypasses the YAML configuration file and directly applies the specified settings to the SLURM job submission.

In [ ]:
slurm.job(exclusive=True, cores=32, memory="200 GB", queue = "interactive", walltime='8:00:00', jobs=1)

### 3.3. Modifying the YAML Configuration File (Configuration Management)

Users directly edit the YAML configuration file `AQUA/aqua/slurm/config-slurm.yml` to modify existing entries or add new sections for different machines. This may involve changing memory limits, walltime, cores, and other job-related parameters, or completely adding a new set of configurations for a newly available machine.


In [28]:
"""
machines:
  existing_machine:
    memory: '32 GB'
    cores: 4
    walltime: '04:00:00'
  new_machine:
    queue: 'name_of_queue'
    account: 'your_project_name'
    memory: '64 GB'
    cores: 8
    walltime: '24:00:00'
    loglevel: 'INFO'
    path_to_output: '/path/to/new_machine/output'
"""

"\nmachines:\n  existing_machine:\n    memory: '32 GB'\n    cores: 4\n    walltime: '04:00:00'\n  new_machine:\n    queue: 'name_of_queue'\n    account: 'your_project_name'\n    memory: '64 GB'\n    cores: 8\n    walltime: '24:00:00'\n    loglevel: 'INFO'\n    path_to_output: '/path/to/new_machine/output'\n"

## 4. Canceling the Slurm Job

### 4.1 Cancelling all jobs of user

In [ ]:
slurm.scancel()

### 4.2 Canceling specific job

Knowing the Job_ID, you can cancel your Job in the queue. For exaple, you can find your Job_ID using the function `squeue().` 

In [ ]:
Job_ID = 4929434
slurm.scancel(Job_ID)

Checking the status of Jobs

In [ ]:
slurm.squeue()

JOBID      CPUS  NODES ST         NAME                 TIME       START_TIME           DEPENDENCY           PARTITION            MIN_MEMORY          


0